In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as tgrad
import torch.nn.functional as F


import os
import time
import tqdm
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from timeit import default_timer as timer

import importlib
import utils
import networks

In [3]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())
print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

True
cuda


# Hyperparameters and Data Sampling
Here in our case, the system is European Call Option PDE and the physical information about the system consists of Boundary Value conditions, final Value conditions and the PDE itself.

In [4]:
K = 10
r = 0.035
sigma = 0.2
T = 1
S_range = [0, int(5*K)]
t_range = [0, T]
gs = lambda x: np.fmax(x-K, 0)
M = 100
N = 5000

lossFunction = nn.MSELoss()
sizes=[2, 50, 50, 50, 50, 50, 50, 50, 50, 1]
lr = 3e-5
activation = 'relu'
loss_weights = [1, 1, 1]

In [5]:
samples = {"pde": 5000, "bc":500, "fc":500}

# sample data generated by finite difference method
X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor = utils.fdm_data(S_range[-1], T, M, N, "500000sample.csv", device)

## PINN with Different Loss weights

In [5]:
weights = [[0.5, 0.25, 0.25], [0.25, 0.5, 0.25], [0.25, 0.25, 0.5], [0.7, 0.15, 0.15], [0.15, 0.7, 0.15], [0.15, 0.15, 0.7]]

In [ ]:
models = []
# Initialize lists to store the loss histories for all components
all_mse_loss_hist = []
all_pde_loss_hist = []
all_bc_loss_hist = []
all_data_loss_hist = []


n_epochs = 5000
# train models with different weights
for i in range(len(weights)):
    w1 = weights[i][0]
    w2 = weights[i][1]
    w3 = weights[i][2]
    wpinn, min_model, mse_loss_hist, pde_loss_hist, bc_loss_hist, data_loss_hist = utils.network_training(
        K, r, sigma, T, S_range[-1], S_range, t_range, gs, samples['bc'], samples['fc'], samples['pde'], RNG_key=123,
        device=device, net='pinn', sizes=sizes, activation='relu', learning_rate=lr, aw_learning_rate=lr, n_epochs=n_epochs, 
        lossFunction=lossFunction, dropout_rate=None, adaptive_rate=None, adaptive_rate_scaler=None, loss_weights=weights[i], adaptive_weight=None, X_train_tensor=X_train_tensor, y_train_tensor=y_train_tensor,
        )
    torch.save(wpinn.state_dict(), f"weight-test/pinn/{w1}-{w2}-{w3}.pth")  # Save the model's state dictionary
    # Save the training loss histories for all components as CSV files
    loss_df = pd.DataFrame({
        'MSE_Loss': mse_loss_hist,
        'PDE_Loss': pde_loss_hist,
        'BC_Loss': bc_loss_hist,
        'Data_Loss': data_loss_hist
    })
    loss_df.to_csv(f'weight-loss/pinn/{w1}-{w2}-{w3}_loss.csv', index=False)
    pass

# IPINN with Different loss weights

In [ ]:
models = []
# Initialize lists to store the loss histories for all components
all_mse_loss_hist = []
all_pde_loss_hist = []
all_bc_loss_hist = []
all_data_loss_hist = []

# train models with different weights
for i in range(len(weights)):
    w1 = weights[i][0]
    w2 = weights[i][1]
    w3 = weights[i][2]
    wipinn, min_model, mse_loss_hist, pde_loss_hist, bc_loss_hist, data_loss_hist = utils.network_training(
        K, r, sigma, T, S_range[-1], S_range, t_range, gs, samples['bc'], samples['fc'], samples['pde'], RNG_key=123,
        device=device, net='ipinn', sizes=sizes, activation='relu', learning_rate=lr, aw_learning_rate=lr, n_epochs=n_epochs, 
        lossFunction=lossFunction, dropout_rate=0, adaptive_rate=0.1, adaptive_rate_scaler=10.0, loss_weights=weights[i], adaptive_weight=None, X_train_tensor=X_train_tensor, y_train_tensor=y_train_tensor,
        )
    torch.save(wipinn.state_dict(), f"weight-test/ipinn/{w1}-{w2}-{w3}.pth")  # Save the model's state dictionary
    # Save the training loss histories for all components as CSV files
    loss_df = pd.DataFrame({
        'MSE_Loss': mse_loss_hist,
        'PDE_Loss': pde_loss_hist,
        'BC_Loss': bc_loss_hist,
        'Data_Loss': data_loss_hist
    })
    loss_df.to_csv(f'weight-loss/ipinn/{w1}-{w2}-{w3}_loss.csv', index=False)
    pass

# Test Error

In [ ]:
# model1 = networks.FeedforwardNeuralNetwork(2, 50, 1, 8)
# model1.to(device)
# model1.load_state_dict(torch.load('default/pinn.pth'))
# model1.eval()
# prediction = model1(X_test_tensor)
# print(lossFunction(prediction, y_test_tensor).item())

In [ ]:
# model1.load_state_dict(torch.load('default/awpinn.pth'))
# model1.eval()
# prediction = model1(X_test_tensor)
# print(lossFunction(prediction, y_test_tensor).item())

In [ ]:
# model1.load_state_dict(torch.load('default/nn.pth'))
# model1.eval()
# prediction = model1(X_test_tensor)
# print(lossFunction(prediction, y_test_tensor).item())

In [ ]:
# sizes=[2, 50, 50, 50, 50, 50, 50, 50, 50, 1]
# model2 = networks.ImprovedNeuralNetwork(sizes, 'relu', 0, 0.1, 10.0)
# model2.to(device)
# model2.load_state_dict(torch.load('default/ipinn.pth'))
# model2.eval()
# prediction = model2(X_test_tensor)
# print(lossFunction(prediction, y_test_tensor).item())

In [ ]:
# model2.load_state_dict(torch.load('default/awipinn.pth'))
# model2.eval()
# prediction = model2(X_test_tensor)
# print(lossFunction(prediction, y_test_tensor).item())

# PINN vs IPINN vs AWPINN

In [6]:
n_epochs = 10000

In [7]:
# Initialize a list to store the loss histories
all_mse_loss_hist = []
all_pde_loss_hist = []
all_bc_loss_hist = []
all_data_loss_hist = []

for i in range(10):
    pinn, min_model, mse_loss_hist, pde_loss_hist, bc_loss_hist, data_loss_hist = utils.network_training(
        K, r, sigma, T, S_range[-1], S_range, t_range, gs, samples['bc'], samples['fc'], samples['pde'], RNG_key=123,
        device=device, net='pinn', sizes=sizes, activation=activation, learning_rate=lr, aw_learning_rate=lr, n_epochs=n_epochs, lossFunction=lossFunction, dropout_rate=0, adaptive_rate=None, adaptive_rate_scaler=0, loss_weights=loss_weights, adaptive_weight=None, X_train_tensor=X_train_tensor, y_train_tensor=y_train_tensor,
        )
    # Save the model's state dictionary
    torch.save(min_model, f"Trained_model/pinn/model_{i}.pth")

    # Save the training loss histories for all components as CSV files
    loss_df = pd.DataFrame({
        'MSE_Loss': mse_loss_hist,
        'PDE_Loss': pde_loss_hist,
        'BC_Loss': bc_loss_hist,
        'Data_Loss': data_loss_hist
    })
    loss_df.to_csv(f'Training_loss/pinn/loss_{i}.csv', index=False)

    # Append the loss histories to the respective lists
    all_mse_loss_hist.append(mse_loss_hist)
    all_pde_loss_hist.append(pde_loss_hist)
    all_bc_loss_hist.append(bc_loss_hist)
    all_data_loss_hist.append(data_loss_hist)
    pass

# Calculate the average losses among all 10 training sessions for each component
average_mse_loss = pd.DataFrame(all_mse_loss_hist).mean(axis=0)
average_pde_loss = pd.DataFrame(all_pde_loss_hist).mean(axis=0)
average_bc_loss = pd.DataFrame(all_bc_loss_hist).mean(axis=0)
average_data_loss = pd.DataFrame(all_data_loss_hist).mean(axis=0)

# Save the average losses as CSV files
average_loss_df = pd.DataFrame({
    'Average_MSE_Loss': average_mse_loss,
    'Average_PDE_Loss': average_pde_loss,
    'Average_BC_Loss': average_bc_loss,
    'Average_Data_Loss': average_data_loss
})
average_loss_df.to_csv('Thesis/pinn_average_losses.csv', index=False)

[Training procedure]: 100%|##########| 10000/10000 [02:37<00:00, 63.65it/s]


In [8]:
# Initialize a list to store the loss histories
all_mse_loss_hist = []
all_pde_loss_hist = []
all_bc_loss_hist = []
all_data_loss_hist = []

for i in range(10):
    ipinn, min_model, mse_loss_hist, pde_loss_hist, bc_loss_hist, data_loss_hist = utils.network_training(
        K, r, sigma, T, S_range[-1], S_range, t_range, gs, samples['bc'], samples['fc'], samples['pde'], RNG_key=123,
        device=device, net='ipinn', sizes=sizes, activation=activation, learning_rate=lr, aw_learning_rate=lr, n_epochs=n_epochs, lossFunction=lossFunction, dropout_rate =0, adaptive_rate=0.1, adaptive_rate_scaler=10.0, loss_weights=loss_weights, adaptive_weight=None, X_train_tensor=X_train_tensor, y_train_tensor=y_train_tensor,
        )
    
    # Save the model's state dictionary
    torch.save(min_model, f"Trained_model/ipinn/model_{i}.pth")

    # Save the training loss histories for all components as CSV files
    loss_df = pd.DataFrame({
        'MSE_Loss': mse_loss_hist,
        'PDE_Loss': pde_loss_hist,
        'BC_Loss': bc_loss_hist,
        'Data_Loss': data_loss_hist
    })
    loss_df.to_csv(f'Training_loss/ipinn/loss_{i}.csv', index=False)

    # Append the loss histories to the respective lists
    all_mse_loss_hist.append(mse_loss_hist)
    all_pde_loss_hist.append(pde_loss_hist)
    all_bc_loss_hist.append(bc_loss_hist)
    all_data_loss_hist.append(data_loss_hist)
    pass

# Calculate the average losses among all 10 training sessions for each component
average_mse_loss = pd.DataFrame(all_mse_loss_hist).mean(axis=0)
average_pde_loss = pd.DataFrame(all_pde_loss_hist).mean(axis=0)
average_bc_loss = pd.DataFrame(all_bc_loss_hist).mean(axis=0)
average_data_loss = pd.DataFrame(all_data_loss_hist).mean(axis=0)

# Save the average losses as CSV files
average_loss_df = pd.DataFrame({
    'Average_MSE_Loss': average_mse_loss,
    'Average_PDE_Loss': average_pde_loss,
    'Average_BC_Loss': average_bc_loss,
    'Average_Data_Loss': average_data_loss
})
average_loss_df.to_csv('Thesis/ipinn_average_losses.csv', index=False)

[Training procedure]: 100%|##########| 10000/10000 [03:36<00:00, 46.13it/s]


In [9]:
# Initialize a list to store the loss histories
all_mse_loss_hist = []
all_pde_loss_hist = []
all_bc_loss_hist = []
all_data_loss_hist = []

for i in range(10):
    awpinn, min_model, mse_loss_hist, pde_loss_hist, bc_loss_hist, data_loss_hist = utils.network_training(
        K, r, sigma, T, S_range[-1], S_range, t_range, gs, samples['bc'], samples['fc'], samples['pde'], RNG_key=123,
        device=device, net='pinn', sizes=sizes, activation=activation, learning_rate=lr, aw_learning_rate=lr, n_epochs=n_epochs, lossFunction=lossFunction, dropout_rate=0, adaptive_rate=None, adaptive_rate_scaler=0, loss_weights=loss_weights, adaptive_weight=True, X_train_tensor=X_train_tensor, y_train_tensor=y_train_tensor,
        )

    # Save the model's state dictionary
    torch.save(min_model, f"Trained_model/awpinn/model_{i}.pth")

    # Save the training loss histories for all components as CSV files
    loss_df = pd.DataFrame({
        'MSE_Loss': mse_loss_hist,
        'PDE_Loss': pde_loss_hist,
        'BC_Loss': bc_loss_hist,
        'Data_Loss': data_loss_hist
    })
    loss_df.to_csv(f'Training_loss/awpinn/loss_{i}.csv', index=False)

    # Append the loss histories to the respective lists
    all_mse_loss_hist.append(mse_loss_hist)
    all_pde_loss_hist.append(pde_loss_hist)
    all_bc_loss_hist.append(bc_loss_hist)
    all_data_loss_hist.append(data_loss_hist)
    pass

# Calculate the average losses among all 10 training sessions for each component
average_mse_loss = pd.DataFrame(all_mse_loss_hist).mean(axis=0)
average_pde_loss = pd.DataFrame(all_pde_loss_hist).mean(axis=0)
average_bc_loss = pd.DataFrame(all_bc_loss_hist).mean(axis=0)
average_data_loss = pd.DataFrame(all_data_loss_hist).mean(axis=0)

# Save the average losses as CSV files
average_loss_df = pd.DataFrame({
    'Average_MSE_Loss': average_mse_loss,
    'Average_PDE_Loss': average_pde_loss,
    'Average_BC_Loss': average_bc_loss,
    'Average_Data_Loss': average_data_loss
})
average_loss_df.to_csv('Thesis/awpinn_average_losses.csv', index=False)

[Training procedure]: 100%|##########| 10000/10000 [02:30<00:00, 66.40it/s]


In [10]:
# Initialize a list to store the loss histories
all_mse_loss_hist = []
all_pde_loss_hist = []
all_bc_loss_hist = []
all_data_loss_hist = []

for i in range(10):
    awipinn, min_model, mse_loss_hist, pde_loss_hist, bc_loss_hist, data_loss_hist = utils.network_training(
        K, r, sigma, T, S_range[-1], S_range, t_range, gs, samples['bc'], samples['fc'], samples['pde'], RNG_key=123,
        device=device, net='ipinn', sizes=sizes, activation=activation, learning_rate=lr, aw_learning_rate=lr, n_epochs=n_epochs, lossFunction=lossFunction, dropout_rate=0, adaptive_rate=0.1, adaptive_rate_scaler=10.0, loss_weights=loss_weights, adaptive_weight=True, X_train_tensor=X_train_tensor, y_train_tensor=y_train_tensor,
        )
    # Save the model's state dictionary
    torch.save(min_model, f"Trained_model/awipinn/model_{i}.pth")

    # Save the training loss histories for all components as CSV files
    loss_df = pd.DataFrame({
        'MSE_Loss': mse_loss_hist,
        'PDE_Loss': pde_loss_hist,
        'BC_Loss': bc_loss_hist,
        'Data_Loss': data_loss_hist
    })
    loss_df.to_csv(f'Training_loss/awipinn/loss_{i}_-3.csv', index=False)

    # Append the loss histories to the respective lists
    all_mse_loss_hist.append(mse_loss_hist)
    all_pde_loss_hist.append(pde_loss_hist)
    all_bc_loss_hist.append(bc_loss_hist)
    all_data_loss_hist.append(data_loss_hist)
    pass

# Calculate the average losses among all 10 training sessions for each component
average_mse_loss = pd.DataFrame(all_mse_loss_hist).mean(axis=0)
average_pde_loss = pd.DataFrame(all_pde_loss_hist).mean(axis=0)
average_bc_loss = pd.DataFrame(all_bc_loss_hist).mean(axis=0)
average_data_loss = pd.DataFrame(all_data_loss_hist).mean(axis=0)

# Save the average losses as CSV files
average_loss_df = pd.DataFrame({
    'Average_MSE_Loss': average_mse_loss,
    'Average_PDE_Loss': average_pde_loss,
    'Average_BC_Loss': average_bc_loss,
    'Average_Data_Loss': average_data_loss
})
average_loss_df.to_csv('Thesis/awipinn_average_losses.csv', index=False)

[Training procedure]: 100%|##########| 10000/10000 [03:36<00:00, 46.10it/s]


In [ ]:
import datetime
net = networks.FeedforwardNeuralNetwork(2, 50, 1, 8)
net.to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-5)
loss_hist6 = []
logging.info(f'{net}\n')
logging.info(f'Training started at {datetime.datetime.now()}\n')
min_train_loss = float("inf")  # Initialize with a large value
final_model = None
start_time = timer()
for _ in tqdm.tqdm(range(n_epochs), desc='[Training procedure]', ascii=True, total=n_epochs):
    prediction = net(X_train_tensor)
    loss = lossFunction(prediction, y_train_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss_hist6.append(loss.item())
    
    if loss.item() < min_train_loss:
        min_train_loss = loss.item()
        final_model = net.state_dict()
    pass

torch.save(final_model, "default/nn.pth")  # Save the model's state dictionary
# Save the training loss history as a CSV file
loss_df = pd.DataFrame(loss_hist6)
loss_df.to_csv('default/nn_loss.csv', index=False)

# Test AWPINN with Different Learning Rate

In [ ]:
# Initialize a list to store the loss histories
n_epochs = 5000
lr_rate_list = [lr/10000, lr/100, lr/10, lr, lr*10, lr*100]


for i in range(len(lr_rate_list)):
    # Initialize lists to store the loss histories for all components
    weight_lr = lr_rate_list[i]
    all_mse_loss_hist = []
    all_pde_loss_hist = []
    all_bc_loss_hist = []
    all_data_loss_hist = []
    
    # run 10 times for each learning rate
    for j in range(10):
        awipinn, min_model, mse_loss_hist, pde_loss_hist, bc_loss_hist, data_loss_hist = utils.network_training(
            K, r, sigma, T, S_range[-1], S_range, t_range, gs, samples['bc'], samples['fc'], samples['pde'], RNG_key=123,
            device=device, net='ipinn', sizes=sizes, activation=activation, learning_rate=lr, aw_learning_rate=weight_lr, n_epochs=n_epochs, lossFunction=lossFunction, dropout_rate=0, adaptive_rate=0.1, adaptive_rate_scaler=10.0, loss_weights=loss_weights, adaptive_weight=True, X_train_tensor=X_train_tensor, y_train_tensor=y_train_tensor,
            )
        # Save the model's state dictionary
        torch.save(min_model, f"Trained_model/awipinn/learning_rate/{weight_lr}/model/{j}.pth")

        # Save the training loss histories for all components as CSV files
        loss_df = pd.DataFrame({
            'MSE_Loss': mse_loss_hist,
            'PDE_Loss': pde_loss_hist,
            'BC_Loss': bc_loss_hist,
            'Data_Loss': data_loss_hist
        })
        loss_df.to_csv(f'Training_loss/awipinn/learning_rate/{weight_lr}/loss/{j}.csv', index=False)

        # Append the loss histories to the respective lists
        all_mse_loss_hist.append(mse_loss_hist)
        all_pde_loss_hist.append(pde_loss_hist)
        all_bc_loss_hist.append(bc_loss_hist)
        all_data_loss_hist.append(data_loss_hist)
        pass

    # Calculate the average losses among all 10 training sessions for each learning rate
    average_mse_loss = pd.DataFrame(all_mse_loss_hist).mean(axis=0)
    average_pde_loss = pd.DataFrame(all_pde_loss_hist).mean(axis=0)
    average_bc_loss = pd.DataFrame(all_bc_loss_hist).mean(axis=0)
    average_data_loss = pd.DataFrame(all_data_loss_hist).mean(axis=0)

    # Save the average losses as CSV files
    average_loss_df = pd.DataFrame({
        'Average_MSE_Loss': average_mse_loss,
        'Average_PDE_Loss': average_pde_loss,
        'Average_BC_Loss': average_bc_loss,
        'Average_Data_Loss': average_data_loss
    })
    average_loss_df.to_csv('test_learning_rate/awipinn/learning_rate/{weight_lr}/average_loss.csv', index=False)